In [ ]:
# to use virtual env
# python3 -m venv venv
# source venv/bin/activate

!pip3 install --upgrade pip
!pip3 install ultralytics opencv-python
!pip3 install pandas


In [ ]:
# download the dataset: https://www.kaggle.com/datasets/andrewmvd/car-plate-detection?resource=download
# rename the folder: "dataset"

In [2]:
import os
import shutil
import random
import time
import pandas as pd
from ultralytics import YOLO
import xml.etree.ElementTree as ET
import torch


In [33]:
# process the annotations in YOLO text format
# split the data into train 70%, val 20% and test 10%

random.seed(42)

image_dir = 'dataset/images'
label_dir = 'dataset/annotations'

out_base = 'dataset_split'

def convert_xml_to_yolo(xml_path, txt_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    size = root.find('size')
    width = float(size.find('width').text)
    height = float(size.find('height').text)

    lines = []
    for obj in root.findall('object'):
        class_id = 0 

        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)

        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        w = (xmax - xmin) / width
        h = (ymax - ymin) / height

        lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

    with open(txt_path, 'w') as f:
        f.write("\n".join(lines))

image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
base_names = [os.path.splitext(f)[0] for f in image_files]

random.shuffle(base_names)
total = len(base_names)
train_end = int(0.7 * total)
val_end = int(0.9 * total)

splits = {
    'train': base_names[:train_end],
    'val': base_names[train_end:val_end],
    'test': base_names[val_end:]
}

for split in ['train', 'val', 'test']:
    os.makedirs(f'{out_base}/images/{split}', exist_ok=True)
    os.makedirs(f'{out_base}/labels/{split}', exist_ok=True)

for split, names in splits.items():
    for name in names:
        img_src = os.path.join(image_dir, f'{name}.png')
        lbl_src = os.path.join(label_dir, f'{name}.xml')

        img_dst = os.path.join(out_base, 'images', split, f'{name}.png')
        txt_dst = os.path.join(out_base, 'labels', split, f'{name}.txt')

        if os.path.exists(img_src):
            shutil.copy(img_src, img_dst)

        if os.path.exists(lbl_src):
            convert_xml_to_yolo(lbl_src, txt_dst)

print("Files split, grouped, and labels converted to YOLO TXT format")


Files split, grouped, and labels converted to YOLO TXT format


In [34]:
if torch.backends.mps.is_available():
    device = "mps"

elif torch.cuda.is_available():        
    device = "cuda"

else:
    device = "cpu"

print(f"Using device: {device}")


Using device: cuda


In [35]:
models = ["yolov8n.pt", "yolov8s.pt", "yolov8m.pt"]
data_yaml = "dataset_split/data.yaml"
epochs = 5

results = []

for model_name in models:
    print(f"\nTraining {model_name}...")
    model = YOLO(model_name)
    
    start_time = time.time()
    metrics = model.train(data=data_yaml, epochs=epochs)
    end_time = time.time()
    train_time = round(end_time - start_time, 2)
    
    map50 = metrics.results_dict.get("metrics/mAP50(B)", None)
    recall = metrics.results_dict.get("metrics/recall(B)", None)
    precision = metrics.results_dict.get("metrics/precision(B)", None)
    f1 = metrics.results_dict.get("metrics/f1(B)", None)
    
    results.append({
        "Model": model_name,
        "Precision": round(precision, 4) if precision else None,
        "Recall": round(recall, 4) if recall else None,
        "F1 Score": round(f1, 4) if f1 else None,
        "mAP50": round(map50, 4) if map50 else None,
        "Training Time (s)": train_time
    })

df = pd.DataFrame(results)
df.to_csv("yolo_model_comparison.csv", index=False)
display(df)


KeyboardInterrupt: 

In [39]:
# train and validate model
# from the table above I could see yolo v8 small performs better

model = YOLO("yolov8s.pt")
model.train(data="dataset_split/data.yaml", name="model_blur",
        epochs=50,
        batch=8,
        imgsz=640,
        lr0=0.001,
        patience=20,
        workers=2,
        verbose=True,
        device = device
        )
metrics = model.val(data="dataset_split/data.yaml")
print(metrics)


Ultralytics 8.3.174  Python-3.9.2 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_split/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=model_blur4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=True, po

train: Scanning D:\Term 2\Deep_Learning_Project\dl_project\dataset_split\labels\train.cache... 1829 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1829/1829 [00:00<?, ?it/s]

train: D:\Term 2\Deep_Learning_Project\dl_project\dataset_split\images\train\car-wbs-MH03AR5549_00000.jpg: corrupt JPEG restored and saved


val: Fast image access  (ping: 0.10.0 ms, read: 408.5101.4 MB/s, size: 393.5 KB)


val: Scanning D:\Term 2\Deep_Learning_Project\dl_project\dataset_split\labels\val.cache... 255 images, 0 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]


...


50 epochs completed in 1.178 hours.
Optimizer stripped from runs\detect\model_blur4\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\model_blur4\weights\best.pt, 22.5MB

Validating runs\detect\model_blur4\weights\best.pt...
Ultralytics 8.3.174  Python-3.9.2 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:03<00:00,  5.04it/s]


                   all        255        262      0.988      0.949      0.985      0.796
Speed: 0.3ms preprocess, 6.0ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs\detect\model_blur4
Ultralytics 8.3.174  Python-3.9.2 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1510.5429.1 MB/s, size: 234.4 KB)


val: Scanning D:\Term 2\Deep_Learning_Project\dl_project\dataset_split\labels\val.cache... 255 images, 0 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:03<00:00,  8.67it/s]


                   all        255        262      0.984      0.949      0.985      0.797
Speed: 0.4ms preprocess, 9.7ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\model_blur42
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002450CAFD490>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032

In [43]:
model = YOLO("runs/detect/model_blur4/weights/best.pt")
results = model.predict(source="test.mp4", conf=0.4, save=True)



WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1076) d:\Term 2\Deep_Learning_Project\dl_project\test.mp4: 640x384 (no detections), 39.3ms
video 1/1 (frame 2/1076) d:\Term 2\Deep_Learning_Project\dl_project\test.mp4: 640x384 (no detections), 11.0ms
video 1/1 (frame 3/1076) d:\Term 2\Deep_Learning_Project\dl_project\test.mp4: 640x384 (no detections), 10.1ms
video 1/1 (frame 4/1076) d:\Term 2\Deep_Learning_Project\dl_project\test.mp4: 640x384 (no detections), 10.1ms
video 1/1 (frame 5/

In [44]:
import cv2
from ultralytics import YOLO

model = YOLO("runs/detect/model_blur4/weights/best.pt")

input_path = "test.mp4"
cap = cv2.VideoCapture(input_path)

fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output.mp4", fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, conf=0.4, stream=True, verbose=False)

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()

        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)
            
            roi = frame[y1:y2, x1:x2]
            if roi.size > 0:
                blur = cv2.GaussianBlur(roi, (35, 35), 0)
                frame[y1:y2, x1:x2] = blur

    out.write(frame)

cap.release()
out.release()
print("Blurred video saved as output.mp4")


Blurred video saved as output.mp4
